In [1]:
import utils

import os, shutil
from datetime import datetime

import sqlalchemy
from sqlalchemy import create_engine, types
from sqlalchemy.dialects.mysql import LONGTEXT

import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [2]:
hostname="localhost"
dbname="apptio"
uname="sreejith"
pwd="password"

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=dbname, user=uname, pw=pwd))

### Clean the user table
---

In [3]:
df = pd.read_sql_query("select * from users" , engine)
df.drop_duplicates(inplace=True)

In [4]:
connection = engine.raw_connection()
cursor = connection.cursor()

In [5]:
df.head()

,user_agent,x_forwarded_for
0,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,69.207.104.248
1,Mozilla/5.0 (X11; CrOS armv7l 7647.84.0) Apple...,99.37.4.61
2,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,68.205.188.176
3,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,199.94.70.33
4,Mozilla/5.0 (X11; CrOS x86_64 8350.68.0) Apple...,50.150.90.52


In [6]:
from user_agents import parse



def create_user_info_map(ua_string):
    
    user_agent = parse(ua_string)
    return [user_agent.browser[0], 
            user_agent.os[0],
            user_agent.os.version,
            user_agent.os.version_string,
            user_agent.device[0],
            user_agent.device.family,
            user_agent.device.brand,
            user_agent.device.model,
            user_agent.is_mobile,
            user_agent.is_tablet,
            user_agent.is_pc
           ]

In [7]:
df["user_info"] = df["user_agent"].progress_apply(create_user_info_map)

100%|██████████| 25306/25306 [00:13<00:00, 1816.36it/s]


In [8]:
df["Browser"] = df["user_info"].apply(lambda x: x[0])
df["OS"] = df["user_info"].apply(lambda x: x[1])
df["OS_Version_String"] = df["user_info"].apply(lambda x: x[3])

df["Device"] = df["user_info"].apply(lambda x: x[4])
df["Device_Family"] = df["user_info"].apply(lambda x: x[5])
df["Device_Brand"] = df["user_info"].apply(lambda x: x[6])
df["Device_Model"] = df["user_info"].apply(lambda x: x[7])

df["Is Mobile"] = df["user_info"].apply(lambda x: x[8])
df["Is Tablet"] = df["user_info"].apply(lambda x: x[9])
df["Is PC"] = df["user_info"].apply(lambda x: x[10])

In [9]:
del df["user_info"]
del df["user_agent"]

In [10]:
df.head()

,x_forwarded_for,Browser,OS,OS_Version_String,Device,Device_Family,Device_Brand,Device_Model,Is Mobile,Is Tablet,Is PC
0,69.207.104.248,Chrome,Windows,XP,Other,Other,None,None,False,False,True
1,99.37.4.61,Chrome,Chrome OS,7647.84.0,Other,Other,None,None,False,False,True
2,68.205.188.176,Chrome,Windows,10,Other,Other,None,None,False,False,True
3,199.94.70.33,Chrome,Windows,7,Other,Other,None,None,False,False,True
4,50.150.90.52,Chrome,Chrome OS,8350.68.0,Other,Other,None,None,False,False,True


In [11]:
table = "users_clean"
command = "DROP TABLE IF EXISTS {};".format(table)
cursor.execute(command)

user_type_map = {x:types.TEXT for x in df.columns.tolist()}
df.to_sql(name=table, con=engine, dtype=user_type_map, index=False, if_exists='append')

----